In [ ]:
import os
import iris
import numpy as np
import iris
import iris.analysis
import iris.quickplot as qplt
import iris.coord_categorisation
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import geopandas as gpd
import iris.plot as iplt
import basic_info

shapefile = gpd.read_file(basic_info.shapefile_file)
shapefile_outline = gpd.read_file(basic_info.shapefile_outline)
print('Shapefile loaded')

output_folder = f'{basic_info.directory_filepath}/outputs'

def add_day_month_season_year_dims(cube):
    # Add day of month
    try:
        cube.remove_coord('day_of_month')
    except iris.exceptions.CoordinateNotFoundError:
        pass
    iris.coord_categorisation.add_day_of_month(cube, 'time', name='day_of_month')
    
    # Add month
    try:
        cube.remove_coord('month')
    except iris.exceptions.CoordinateNotFoundError:
        pass
    iris.coord_categorisation.add_month(cube, 'time', name='month')
    
    # Add season year
    try:
        cube.remove_coord('season_year')
    except iris.exceptions.CoordinateNotFoundError:
        pass
    iris.coord_categorisation.add_season_year(cube, 'time', name='season_year')
    
    return cube

In [ ]:
# Loading baseline data
CMIP6_baseline = iris.load_cube(basic_info.projections_cmip6_baseline_rh)
CORDEX_baseline = iris.load_cube(basic_info.projections_cordex_baseline_rh)

add_day_month_season_year_dims(CMIP6_baseline)
add_day_month_season_year_dims(CORDEX_baseline)

# Masking the baseline data
CMIP6_baseline = iris.util.mask_cube_from_shapefile(CMIP6_baseline, shapefile_outline.geometry[0], minimum_weight=0.5)
CORDEX_baseline = iris.util.mask_cube_from_shapefile(CORDEX_baseline, shapefile_outline.geometry[0], minimum_weight=0.5)

In [ ]:
CMIP6_2041_2060_cubelist = iris.cube.CubeList([])
CMIP6_2080_2099_cubelist = iris.cube.CubeList([])

CORDEX_2041_2060_cubelist = iris.cube.CubeList([])
CORDEX_2080_2099_cubelist = iris.cube.CubeList([])

filepaths = [
    f'{basic_info.rh_projections_data_folder}/CMIP6/2041_2060',
    f'{basic_info.rh_projections_data_folder}/CMIP6/2080_2099',
    f'{basic_info.rh_projections_data_folder}/CORDEX/2041_2060',
    f'{basic_info.rh_projections_data_folder}/CORDEX/2080_2099'
]

cubelists = [
    CMIP6_2041_2060_cubelist,
    CMIP6_2080_2099_cubelist,
    CORDEX_2041_2060_cubelist,
    CORDEX_2080_2099_cubelist
]

def load_and_append_projection_cubes(filepath, cubelist):
    files = []
    try:
        files = os.listdir(filepath)
        print(f"Files in {filepath} loaded")
    except Exception as e:
        print(f"Error loading files in {filepath}: {e}")
    for file in files:
        try:
            cube = iris.load_cube(f'{filepath}/{file}')
            add_day_month_season_year_dims(cube)
            # cube.convert_units('Kelvin')
            cube = iris.util.mask_cube_from_shapefile(cube, shapefile_outline.geometry[0], minimum_weight=0.5)
            cubelist.append(cube)
        except Exception as e:
            print(f"Error loading cube from {file}: {e}")
    return cubelist

for filepath, cubelist in zip(filepaths, cubelists):
    load_and_append_projection_cubes(filepath, cubelist)

# Maps

## Heat index difference

In [ ]:
CMIP6_baseline_map = CMIP6_baseline.aggregated_by('season_year', iris.analysis.MEAN)
CMIP6_baseline_map = CMIP6_baseline_map.collapsed('time', iris.analysis.MEAN)

CORDEX_baseline_map = CORDEX_baseline.aggregated_by('season_year', iris.analysis.MEAN)
CORDEX_baseline_map = CORDEX_baseline_map.collapsed('time', iris.analysis.MEAN)

def create_average_model_map(cubelist):
    # Calculate the temporal mean of each cube in the CubeList
    # Aggregate each cube by 'season_year' and calculate the mean
    yearly_mean_cubes = iris.cube.CubeList([cube.aggregated_by('season_year', iris.analysis.MEAN) for cube in cubelist])

    # Collapse each cube in the yearly_mean_cubes list over the 'time' dimension to calculate the mean of the daily max values per year
    mean_cubes = iris.cube.CubeList([cube.collapsed('time', iris.analysis.MEAN) for cube in yearly_mean_cubes])

    # Remove conflicting attributes, scalar coordinates, and cell methods from each cube that otherwise cause the merge to fail (found through trial and error with each run outputting new conflicting attributes and alike in the output)
    for cube in mean_cubes:
        cube.attributes = {}
        for coord in ['time', 'day_of_month', 'height', 'month', 'season', 'season_year', 'year', 'day_of_year']:
            if cube.coords(coord):
                cube.remove_coord(coord)
        cube.cell_methods = ()

    # Regrid all cubes to the grid of the first cube in the list
    cube_to_regrid_to = mean_cubes[0]
    regridded_cubes = iris.cube.CubeList([cube.regrid(cube_to_regrid_to, iris.analysis.Linear()) for cube in mean_cubes])

    # Add a new dimension to each cube to 'stack' them
    for i, cube in enumerate(regridded_cubes):
        cube.add_aux_coord(iris.coords.AuxCoord(i, long_name='model_index'))

    # Merge the cubes into a single cube ('stacked')
    stacked_cube = regridded_cubes.merge_cube()

    # Calculate the mean across the new dimension
    average_cube = stacked_cube.collapsed('model_index', iris.analysis.MEAN)

    return average_cube

# Average maps
CMIP6_2041_2060_map = create_average_model_map(CMIP6_2041_2060_cubelist)
CMIP6_2080_2099_map = create_average_model_map(CMIP6_2080_2099_cubelist)

CORDEX_2041_2060_map = create_average_model_map(CORDEX_2041_2060_cubelist)
CORDEX_2080_2099_map = create_average_model_map(CORDEX_2080_2099_cubelist)

# Difference maps
CMIP6_2041_2060_diff = CMIP6_2041_2060_map - CMIP6_baseline_map
CMIP6_2080_2099_diff = CMIP6_2080_2099_map - CMIP6_baseline_map

CORDEX_2041_2060_diff = CORDEX_2041_2060_map - CORDEX_baseline_map
CORDEX_2080_2099_diff = CORDEX_2080_2099_map - CORDEX_baseline_map

# Create a figure with 3 columns and 5 rows
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 23), subplot_kw={'projection': ccrs.PlateCarree()})

# Define the models and their corresponding maps
models = [
    ("CMIP6 2041-2060", CMIP6_baseline_map, CMIP6_2041_2060_map, CMIP6_2041_2060_diff),
    ("CMIP6 2080-2099", CMIP6_baseline_map, CMIP6_2080_2099_map, CMIP6_2080_2099_diff),
    ("CORDEX 2041-2060", CORDEX_baseline_map, CORDEX_2041_2060_map, CORDEX_2041_2060_diff),
    ("CORDEX 2080-2099", CORDEX_baseline_map, CORDEX_2080_2099_map, CORDEX_2080_2099_diff)
]

# Column titles
column_titles = ['Baseline', 'Average', 'Difference']

# Plot each category of maps using a for loop on rows and the contents of the 'models' list
for row, (model_name, baseline_map, average_map, difference_map) in enumerate(models):
    # Plot baseline map
    plt.sca(axes[row, 0])
    baseline_plot = iplt.pcolormesh(baseline_map, axes=axes[row, 0], cmap='Reds')
    shapefile.plot(ax=axes[row, 0], color='none', edgecolor='black', linewidth=1)  # Add shapefile to the plot
    if row == 0:
        axes[row, 0].set_title(column_titles[0], fontsize=16)
    fig.colorbar(baseline_plot, ax=axes[row, 0], orientation='horizontal', pad=0.02)

    # Plot average map
    plt.sca(axes[row, 1])
    average_plot = iplt.pcolormesh(average_map, axes=axes[row, 1], cmap='Reds')
    if row == 0:
        axes[row, 1].set_title(column_titles[1], fontsize=16)
    shapefile.plot(ax=axes[row, 1], color='none', edgecolor='black', linewidth=1)  # Add shapefile to the plot
    fig.colorbar(average_plot, ax=axes[row, 1], orientation='horizontal', pad=0.02)

    # Plot difference map with blue color map
    plt.sca(axes[row, 2])
    difference_plot = iplt.pcolormesh(difference_map, axes=axes[row, 2], cmap='Blues')
    if row == 0:
        axes[row, 2].set_title(column_titles[2], fontsize=16)
    shapefile.plot(ax=axes[row, 2], color='none', edgecolor='black', linewidth=1)  # Add shapefile to the plot
    fig.colorbar(difference_plot, ax=axes[row, 2], orientation='horizontal', pad=0.02)

# Add row labels using fig.text for better control
row_labels = [f'CMIP6 \n 2041-2060', f'CMIP6 \n 2080-2099', f'CORDEX \n 2041-2060', f'CORDEX \n 2080-2099']
for row, label in enumerate(row_labels):
    fig.text(0.085, 0.64 - row * 0.17, label, va='center', ha='center', rotation='vertical', fontsize=14) # Note to self: initial y value is FROM THE BOTTOM

# Adjust layout
plt.subplots_adjust(hspace=-0.65)  # Use a negative value for hspace to reduce the space between subplots
plt.tight_layout(rect=[0.1, 0, 1, 0.95])
plt.savefig(f'{output_folder}/rh_projection_maps.png')

plt.show()

# Barplots

In [ ]:
CMIP6_overall_baseline = CMIP6_baseline.collapsed('time', iris.analysis.MEAN)
CMIP6_overall_baseline = CMIP6_overall_baseline.collapsed(['latitude', 'longitude'], iris.analysis.MEAN)
CORDEX_overall_baseline = CORDEX_baseline.collapsed('time', iris.analysis.MEAN)
CORDEX_overall_baseline = CORDEX_overall_baseline.collapsed(['latitude', 'longitude'], iris.analysis.MEAN)

# Averages
def calculate_overall_mean(cubelist):
    averages = []
    for cube in cubelist:
        cube = cube.collapsed('time', iris.analysis.MEAN)
        cube = cube.collapsed(['latitude', 'longitude'], iris.analysis.MEAN)
        averages.append(cube.data)
    model_average = np.mean(averages)
    print(f"Model average: {model_average}")
    return model_average

CMIP6_2041_2060_average = calculate_overall_mean(CMIP6_2041_2060_cubelist)
CMIP6_2080_2099_average = calculate_overall_mean(CMIP6_2080_2099_cubelist)

CORDEX_2041_2060_average = calculate_overall_mean(CORDEX_2041_2060_cubelist)
CORDEX_2080_2099_average = calculate_overall_mean(CORDEX_2080_2099_cubelist)

# Differences
CMIP6_2041_2060_difference = CMIP6_2041_2060_average - CMIP6_overall_baseline.data
CMIP6_2080_2099_difference = CMIP6_2080_2099_average - CMIP6_overall_baseline.data

CORDEX_2041_2060_difference = CORDEX_2041_2060_average - CORDEX_overall_baseline.data
CORDEX_2080_2099_difference = CORDEX_2080_2099_average - CORDEX_overall_baseline.data

print(f"CMIP6 2041-2060 difference: {CMIP6_2041_2060_difference}")
print(f"CMIP6 2080-2099 difference: {CMIP6_2080_2099_difference}")
print(f"CORDEX 2041-2060 difference: {CORDEX_2041_2060_difference}")
print(f"CORDEX 2080-2099 difference: {CORDEX_2080_2099_difference}")

# Define the models and periods
models = ["CMIP6", "CORDEX"]
periods = ["2041-2060", "2080-2099"]

# Define the data for each model and period
data = {
    "CMIP6": [CMIP6_2041_2060_difference, CMIP6_2080_2099_difference],
    "CORDEX": [CORDEX_2041_2060_difference, CORDEX_2080_2099_difference]
}

# Calculate the number of models and periods to plot, creatd
n_models = len(models)
n_periods = len(periods)

# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Set the width and positions of the bars
bar_width = 0.35

index = np.arange(n_models)

# Plot the bars for each period
for i, period in enumerate(periods):
    values = [data[model][i] for model in models]
    ax.bar(index + i * bar_width, values, bar_width, label=period)

# Set the x-axis labels to the models (otherwise just numbers)
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(models)

# Add labels, title, and legend
ax.set_xlabel('Model')
ax.set_ylabel('Difference (%)')
ax.set_title('Model Differences for 2041-2060 and 2080-2099')
ax.legend()

# Show the plot
plt.savefig(f'{output_folder}/rh_model_differences.png')

plt.show()